In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import seaborn as sn
from lifelines.fitters.kaplan_meier_fitter import KaplanMeierFitter
from lifelines import CoxPHFitter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
import lifelines

pd.set_option('display.max_columns', None)


In [83]:
df = pd.read_csv("brca_metabric/brca_metabric_clinical_data.tsv", sep="\t")
df.head()

,Study ID,Patient ID,Sample ID,Age at Diagnosis,Type of Breast Surgery,Cancer Type,Cancer Type Detailed,Cellularity,Chemotherapy,Pam50 + Claudin-low subtype,Cohort,ER status measured by IHC,ER Status,Neoplasm Histologic Grade,HER2 status measured by SNP6,HER2 Status,Tumor Other Histologic Subtype,Hormone Therapy,Inferred Menopausal State,Integrative Cluster,Primary Tumor Laterality,Lymph nodes examined positive,Mutation Count,Nottingham prognostic index,Oncotree Code,Overall Survival (Months),Overall Survival Status,PR Status,Radio Therapy,Relapse Free Status (Months),Relapse Free Status,Number of Samples Per Patient,Sample Type,Sex,3-Gene classifier subtype,TMB (nonsynonymous),Tumor Size,Tumor Stage,Patient's Vital Status
0,brca_metabric,MB-0000,MB-0000,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,NO,claudin-low,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Ductal/NST,YES,Post,4ER+,Right,10.0,NaN,6.044,IDC,140.500000,0:LIVING,Negative,YES,138.65,0:Not Recurred,1,Primary,Female,ER-/HER2-,0.000000,22.0,2.0,Living
1,brca_metabric,MB-0002,MB-0002,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,NO,LumA,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Ductal/NST,YES,Pre,4ER+,Right,0.0,2.0,4.020,IDC,84.633333,0:LIVING,Positive,YES,83.52,0:Not Recurred,1,Primary,Female,ER+/HER2- High Prolif,2.615035,10.0,1.0,Living
2,brca_metabric,MB-0005,MB-0005,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,YES,LumB,1.0,Positve,Positive,2.0,NEUTRAL,Negative,Ductal/NST,YES,Pre,3,Right,1.0,2.0,4.030,IDC,163.700000,1:DECEASED,Positive,NO,151.28,1:Recurred,1,Primary,Female,NaN,2.615035,15.0,2.0,Died of Disease
3,brca_metabric,MB-0006,MB-0006,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,YES,LumB,1.0,Positve,Positive,2.0,NEUTRAL,Negative,Mixed,YES,Pre,9,Right,3.0,1.0,4.050,MDLC,164.933333,0:LIVING,Positive,YES,162.76,0:Not Recurred,1,Primary,Female,NaN,1.307518,25.0,2.0,Living
4,brca_metabric,MB-0008,MB-0008,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,YES,LumB,1.0,Positve,Positive,3.0,NEUTRAL,Negative,Mixed,YES,Post,9,Right,8.0,2.0,6.080,MDLC,41.366667,1:DECEASED,Positive,YES,18.55,1:Recurred,1,Primary,Female,ER+/HER2- High Prolif,2.615035,40.0,2.0,Died of Disease


## Prepare Dataframe

In [103]:
df["Censorship"] = df["Patient's Vital Status"] == "Living"
df_clear = df.dropna()
df_clear = df_clear.drop(["Study ID", "Patient ID", "Sample ID","Overall Survival Status", "Patient's Vital Status", "Cancer Type", "Number of Samples Per Patient", "Sex", "Sample Type"
, "Cancer Type Detailed", "Tumor Other Histologic Subtype", "Oncotree Code", "Relapse Free Status", "Relapse Free Status (Months)", "TMB (nonsynonymous)"], axis = 1)
# replace Pam50 + Claudin-low subtype == NC with aNC , "Pam50 + Claudin-low subtype"
df_clear["HER2 status measured by SNP6"] = df_clear["HER2 status measured by SNP6"].replace("UNDEF", "AUNDEF")
df_clear["Pam50 + Claudin-low subtype"] = df_clear["Pam50 + Claudin-low subtype"].replace("NC", "ANC")
df_clear = pd.get_dummies(df_clear, drop_first=True)
df_clear.head()

,Age at Diagnosis,Cohort,Neoplasm Histologic Grade,Lymph nodes examined positive,Mutation Count,Nottingham prognostic index,Overall Survival (Months),TMB (nonsynonymous),Tumor Size,Tumor Stage,Censorship,Type of Breast Surgery_MASTECTOMY,Cellularity_Low,Cellularity_Moderate,Chemotherapy_YES,Pam50 + Claudin-low subtype_Basal,Pam50 + Claudin-low subtype_Her2,Pam50 + Claudin-low subtype_LumA,Pam50 + Claudin-low subtype_LumB,Pam50 + Claudin-low subtype_Normal,Pam50 + Claudin-low subtype_claudin-low,ER status measured by IHC_Positve,ER Status_Positive,HER2 status measured by SNP6_GAIN,HER2 status measured by SNP6_LOSS,HER2 status measured by SNP6_NEUTRAL,HER2 Status_Positive,Hormone Therapy_YES,Inferred Menopausal State_Pre,Integrative Cluster_10,Integrative Cluster_2,Integrative Cluster_3,Integrative Cluster_4ER+,Integrative Cluster_4ER-,Integrative Cluster_5,Integrative Cluster_6,Integrative Cluster_7,Integrative Cluster_8,Integrative Cluster_9,Primary Tumor Laterality_Right,PR Status_Positive,Radio Therapy_YES,3-Gene classifier subtype_ER+/HER2- Low Prolif,3-Gene classifier subtype_ER-/HER2-,3-Gene classifier subtype_HER2+
1,43.19,1.0,3.0,0.0,2.0,4.020,84.633333,2.615035,10.0,1.0,True,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0
4,76.97,1.0,3.0,8.0,2.0,6.080,41.366667,2.615035,40.0,2.0,False,1,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0
5,78.77,1.0,3.0,0.0,4.0,4.062,7.800000,5.230071,31.0,4.0,False,1,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0
10,86.41,1.0,3.0,1.0,4.0,5.032,36.566667,5.230071,16.0,2.0,False,0,0,1,0,0,0,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0
11,84.22,1.0,2.0,0.0,5.0,3.056,36.266667,6.537589,28.0,2.0,False,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Test set

In [104]:
df_train = df_clear.sample(frac=0.8, random_state=0)
df_test = df_clear.drop(df_train.index)    

In [ ]:
plt.figure(figsize=(30, 30))
sn.heatmap(df_train.corr(), annot=True)

In [ ]:
X = df_train.drop(["Overall Survival (Months)", "Censorship"], axis = 1)
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

In [107]:
num_folds =10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=0)
mean_concordance = 0.0
models = []
for i, (train_index, test_index) in enumerate(kf.split(df_clear)):
    train = df_clear.iloc[train_index]
    valid = df_clear.iloc[test_index]

    cph = CoxPHFitter(penalizer=0.01)
    cph.fit(train, duration_col = "Overall Survival (Months)", event_col = "Censorship")
    concordance_index = cph.score(valid, scoring_method="concordance_index")
    print("Fold: ", i, "concordance index: ", concordance_index)
    mean_concordance += concordance_index

    models.append(cph)
print("Mean concordance index: ", mean_concordance / num_folds)


Fold:  0 concordance index:  0.6217391304347826
Fold:  1 concordance index:  0.6925754060324826
Fold:  2 concordance index:  0.7186358099878197
Fold:  3 concordance index:  0.6265133171912833
Fold:  4 concordance index:  0.7341188524590164
Fold:  5 concordance index:  0.7436801881246325
Fold:  6 concordance index:  0.7022716288061865
Fold:  7 concordance index:  0.6490993995997332
Fold:  8 concordance index:  0.7222857142857143
Fold:  9 concordance index:  0.6885669943093636
Mean concordance index:  0.6899486441231015


In [119]:
predictions = np.zeros(df_test.shape[0])
for model in models:
    predictions -= model.predict_partial_hazard(df_test)
predictions /= num_folds
lifelines.utils.concordance_index(df_test["Overall Survival (Months)"], predictions, event_observed=df_test["Censorship"])

0.7328527741371778

In [79]:
cph.score(df_test, scoring_method="concordance_index")

0.6982306684141546

In [53]:
import h5py
from collections import defaultdict
datasets = defaultdict(dict)
dataset_file = "brca_metabric/metabric_IHC4_clinical_train_test.h5"
with h5py.File(dataset_file, 'r') as fp:
       for ds in fp:
           for array in fp[ds]:
               datasets[ds][array] = fp[ds][array][:]
datasets["test"]["x"]

array([[ 8.003323 ,  5.3839517, 13.391568 , ...,  1.       ,  0.       ,
        44.07     ],
       [ 5.877926 ,  8.036838 , 10.16474  , ...,  0.       ,  1.       ,
        47.85     ],
       [ 5.8922563,  6.4251266, 11.480591 , ...,  0.       ,  1.       ,
        55.62     ],
       ...,
       [ 5.90161  ,  5.2722373, 14.20095  , ...,  0.       ,  1.       ,
        57.77     ],
       [ 6.8181086,  5.372744 , 11.652624 , ...,  0.       ,  1.       ,
        58.89     ],
       [ 5.725708 ,  5.4497185,  9.680736 , ...,  0.       ,  0.       ,
        60.63     ]], dtype=float32)

In [68]:
df_binary = pd.DataFrame({"x" : ['a', 'b', 'a'], "y" : ['a', 'b', 'a'], "z" : ['a', 'b', 'c']})
pd.get_dummies(df_binary, drop_first=True)

,x_b,y_b,z_b,z_c
0,0,0,0,0
1,1,1,1,0
2,0,0,0,1
